# Start code

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [4]:

train_dataset = KeypointsDataset("../dataset/data/images","../dataset/data/data_train.json")
val_dataset = KeypointsDataset("../dataset/data/images","../dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Creat model

In [5]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

d:\Programar\tennis-yolo\.tennis-yolo\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Programar\tennis-yolo\.tennis-yolo\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

# Train model

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14452.7919921875
Epoch 0, iter 10, loss: 14215.611328125
Epoch 0, iter 20, loss: 13907.859375
Epoch 0, iter 30, loss: 14155.5341796875
Epoch 0, iter 40, loss: 12804.5439453125
Epoch 0, iter 50, loss: 12729.8388671875
Epoch 0, iter 60, loss: 12041.5732421875
Epoch 0, iter 70, loss: 12200.240234375
Epoch 0, iter 80, loss: 11921.5537109375
Epoch 0, iter 90, loss: 11380.140625
Epoch 0, iter 100, loss: 11205.7314453125
Epoch 0, iter 110, loss: 11372.5224609375
Epoch 0, iter 120, loss: 10396.810546875
Epoch 0, iter 130, loss: 9483.541015625
Epoch 0, iter 140, loss: 9946.8251953125
Epoch 0, iter 150, loss: 9165.2744140625
Epoch 0, iter 160, loss: 8798.0302734375
Epoch 0, iter 170, loss: 9025.5927734375
Epoch 0, iter 180, loss: 11235.076171875
Epoch 0, iter 190, loss: 7956.87744140625
Epoch 0, iter 200, loss: 7418.37060546875
Epoch 0, iter 210, loss: 7593.09326171875
Epoch 0, iter 220, loss: 7115.81787109375
Epoch 0, iter 230, loss: 7395.87255859375
Epoch 0, iter 240, lo

In [ ]:
torch.save(model.stat_dict(), "models/keypoints_model.pth")